<a href="https://colab.research.google.com/github/AbrahamKong/CMPE256-Vector_Space_Assignment/blob/main/CMPE_256_Vector_Space_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import requests

data_dir = "/content/"
data_files = [
    "d1.txt", "d2.txt", "d3.txt",
    "d4.txt"
]

data = pd.Series()
for file in data_files:
    f = open(data_dir + file, 'r')
    file_contents = f.read()
    data[file[:-4]] = file_contents
    f.close()
    print(data[file[:-4]])

User selected Wedding gown.
User ordered on-line rose flowers.
User searched diamond ring.
User selected white wedding gown, online flowers, 3 carat diamond ring.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


In [13]:
from collections import Counter

bag_of_words = (
    data.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words

d1    {'user': 1, 'selected': 1, 'wedding': 1, 'gown...
d2    {'user': 1, 'ordered': 1, 'on': 1, 'line': 1, ...
d3    {'user': 1, 'searched': 1, 'diamond': 1, 'ring...
d4    {'user': 1, 'selected': 1, 'white': 1, 'weddin...
dtype: object

In [14]:
tf = pd.DataFrame(list(bag_of_words))
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0


In [15]:
tf = tf.fillna(0)
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(data) # This determines the vocabulary.
tf_sparse = vec.transform(data)

tf_sparse

<4x15 sparse matrix of type '<class 'numpy.int64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [18]:
tf_sparse.todense()

matrix([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1]])

In [19]:
pd.DataFrame(
    tf_sparse.todense(),
    columns=vec.get_feature_names()
)

,carat,diamond,flowers,gown,line,on,online,ordered,ring,rose,searched,selected,user,wedding,white
0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0
1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0
2,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0
3,1,1,1,1,0,0,1,0,1,0,0,1,1,1,1


In [20]:
set(tf.columns) - set(vec.get_feature_names())

{'3'}

In [22]:
vec = CountVectorizer(ngram_range=(1, 3))
vec.fit(data)
vec.transform(data)

<4x48 sparse matrix of type '<class 'numpy.int64'>'
	with 60 stored elements in Compressed Sparse Row format>

In [23]:
# number of non-zero values in the sparse matrix.
vec.transform(data).count_nonzero()

60

In [24]:
# Get document frequencies 
# (How many documents does each word appear in?)
df = (tf > 0).sum(axis=0)
df

user        4
selected    2
wedding     2
gown        2
ordered     1
on          1
line        1
rose        1
flowers     2
searched    1
diamond     2
ring        2
white       1
online      1
3           1
carat       1
dtype: int64

In [25]:
import numpy as np

# Get IDFs
idf = np.log(len(tf) / df)
idf

user        0.000000
selected    0.693147
wedding     0.693147
gown        0.693147
ordered     1.386294
on          1.386294
line        1.386294
rose        1.386294
flowers     0.693147
searched    1.386294
diamond     0.693147
ring        0.693147
white       1.386294
online      1.386294
3           1.386294
carat       1.386294
dtype: float64

In [26]:
# Calculate TF-IDFs
tf_idf = tf * idf
tf_idf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,1.386294,1.386294,1.386294,1.386294,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000
3,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.693147,1.386294,1.386294,1.386294,1.386294


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(data) # This determines the vocabulary.
tf_idf_sparse = vec.transform(data)
tf_idf_sparse

<4x15 sparse matrix of type '<class 'numpy.float64'>'
	with 24 stored elements in Compressed Sparse Row format>